# Which version of the website should you use?

## 📖 Background
You work for an early-stage startup in Germany. Your team has been working on a redesign of the landing page. The team believes a new design will increase the number of people who click through and join your site. 

They have been testing the changes for a few weeks and now they want to measure the impact of the change and need you to determine if the increase can be due to random chance or if it is statistically significant.

## 💾 The data
The team assembled the following file:

#### Redesign test data
- "treatment" - "yes" if the user saw the new version of the landing page, no otherwise.
- "new_images" - "yes" if the page used a new set of images, no otherwise.
- "converted" - 1 if the user joined the site, 0 otherwise.

The control group is those users with "no" in both columns: the old version with the old set of images.

In [1]:
import pandas as pd
df = pd.read_csv('./data/redesign.csv')
df.head()

,treatment,new_images,converted
0,yes,yes,0
1,yes,yes,0
2,yes,yes,0
3,yes,no,0
4,no,yes,0


In [2]:
df.columns

Index(['treatment', 'new_images', 'converted'], dtype='object')

In [3]:
df.dtypes

treatment     object
new_images    object
converted      int64
dtype: object

In [4]:
# Summary statistics
print(df.groupby(['treatment', 'new_images'])['converted'].mean().reset_index())

  treatment new_images  converted
0        no         no   0.107104
1        no        yes   0.112538
2       yes         no   0.120047
3       yes        yes   0.113724


In [5]:
# Check balance of new_images across treatment groups
print(df.groupby('treatment')['new_images'].value_counts(normalize=True))

treatment  new_images
no         no            0.5
           yes           0.5
yes        no            0.5
           yes           0.5
Name: proportion, dtype: float64


In [11]:
from statsmodels.stats.proportion import proportions_ztest

# Extract data
control_group = df[df['treatment'] == 'no']['converted']
treatment_group = df[df['treatment'] == 'yes']['converted']

# Number of successes and samples
successes = [control_group.sum(), treatment_group.sum()]
samples = [len(control_group), len(treatment_group)]

# Perform the test
z_stat, p_value = proportions_ztest(successes, samples, alternative='two-sided')
print(f"Z-statistic: {z_stat}, P-value: {p_value}")

Z-statistic: -2.241825154497307, P-value: 0.024972674822144957


In [8]:
import statsmodels.api as sm
import pandas as pd

# Convert categorical variables to dummy variables
df['treatment_yes'] = (df['treatment'] == 'yes').astype(int)
df['new_images_yes'] = (df['new_images'] == 'yes').astype(int)

# Define the model
X = df[['treatment_yes', 'new_images_yes']]
X = sm.add_constant(X)  # Add intercept
y = df['converted']

model = sm.Logit(y, X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.353407
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              converted   No. Observations:                40484
Model:                          Logit   Df Residuals:                    40481
Method:                           MLE   Df Model:                            2
Date:                Mon, 10 Feb 2025   Pseudo R-squ.:               0.0001763
Time:                        00:48:29   Log-Likelihood:                -14307.
converged:                       True   LL-Null:                       -14310.
Covariance Type:            nonrobust   LLR p-value:                   0.08020
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.0904      0.027    -76.314      0.000      -2.144      -2.037
treatment_yes    

In [10]:
import numpy as np
odds_ratio = np.exp(model.params['treatment_yes'])
print(f"Odds Ratio for Treatment: {odds_ratio}")

Odds Ratio for Treatment: 1.07284166603891
